# NNPP
This Notebook reimplements the Method of the Paper from RL 18 to compare results to the pytoch geometric model.
This should archieve a CRPS score around 0.78

In [12]:
import sys
sys.path.append('../utils')

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from tqdm import trange

%matplotlib inline
plt.style.use('default')

In [13]:
# read data (can be downloaded from https://doi.org/10.6084/m9.figshare.13516301.v1)

data = pd.read_feather(path = "C:/Users/morit/GNNPP/data_RL18.feather")
data.station = pd.to_numeric(data.station, downcast = 'integer')

# drop soil moisture predictions due to missing values
# Note that this is a minor change compared to the paper, but does not have a significant effect
data = data.drop(['sm_mean', 'sm_var'], axis=1)

# split into train and test data
eval_start = 1626724
train_end = 1626723

train_features_raw = data.iloc[:train_end,3:42].to_numpy()
train_targets = data.iloc[:train_end,2].to_numpy()
train_IDs = data.iloc[:train_end,1].to_numpy()

test_features_raw = data.iloc[eval_start:,3:42].to_numpy()
test_targets = data.iloc[eval_start:,2].to_numpy()
test_IDs = data.iloc[eval_start:,1].to_numpy()

In [14]:
# normalize data

def normalize(data, method=None, shift=None, scale=None):
    result = np.zeros(data.shape)
    if method == "MAX":
        scale = np.max(data, axis=0)
        shift = np.zeros(scale.shape)
    for index in range(len(data[0])):
        result[:,index] = (data[:,index] - shift[index]) / scale[index]
    return result, shift, scale

train_features, train_shift, train_scale = normalize(train_features_raw[:,:], method="MAX")

test_features = normalize(test_features_raw[:,:], shift=train_shift, scale=train_scale)[0]

In [15]:
def crps(mu: torch.tensor, sigma: torch.tensor, y: torch.tensor):
    """Calculates the Continuous Ranked Probability Score (CRPS) assuming normally distributed df

    Args:
        mu (torch.tensor): mean
        sigma (torch.tensor): standard deviation
        y (torch.tensor): observed df

    Returns:
        torch.tensor: CRPS value
    """
    y = y.view((-1,1)) # make sure y has the right shape
    PI = np.pi #3.14159265359
    omega = (y - mu) / sigma
    # PDF of normal distribution at omega
    pdf = 1/(torch.sqrt(torch.tensor(2 * PI))) * torch.exp(-0.5 * omega ** 2)
    
    # Source: https://stats.stackexchange.com/questions/187828/how-are-the-error-function-and-standard-normal-distribution-function-related
    cdf = 0.5 * (1 + torch.erf(omega / torch.sqrt(torch.tensor(2))))
    
    crps = sigma * (omega * (2 * cdf - 1) + 2 * pdf - 1/torch.sqrt(torch.tensor(PI)))
    return  torch.mean(crps)

In [25]:
# Model Definition
class NNPP(nn.Module):
    def __init__(self, INPUT_DIM:int):
        super(NNPP, self).__init__()
        self.emb = nn.Embedding(num_embeddings=535, embedding_dim=2)
        self.lin1 = nn.Linear(in_features=INPUT_DIM+2-1, out_features=512)
        self.lin2 = nn.Linear(in_features=512, out_features=2)
        

    def forward(self, x):
        station_ids = x[:, 0].long()
        emb_station = self.emb(station_ids)
        x = torch.cat((emb_station, x[:, 1:]), dim=1) # Concatenate embedded station_id to rest of the feature vector
        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        mu, sigma = torch.split(x, 1, dim=-1)
        sigma = F.softplus(sigma) # ensure that sigma is positive
        return mu, sigma

In [37]:
trn_scores = []
test_scores = []
preds = []

nreps = 10

def train(epochs:int, train_features, train_labels, test_features, test_labels, model, optimizer, crps, device):
    # Convert numpy arrays to tensors
    train_features = torch.Tensor(train_features).to(device)
    train_labels = torch.Tensor(train_labels).to(device)
    test_features = torch.Tensor(test_features).to(device)
    test_labels = torch.Tensor(test_labels).to(device)
    # Create a TensorDataset
    train_dataset = TensorDataset(train_features, train_labels)
    test_dataset = TensorDataset(test_features, test_labels)
    # Define the batch size
    batch_size = 4096
    # Create a DataLoader
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
    epoch_loss = 0
    test_loss = 0
    # Train
    print("Training...")
    for epoch in range(epochs):
        for batch_features, batch_labels in train_dataloader:
            # Forward pass
            mu, sigma = model(batch_features)
            # Compute the loss
            loss = crps(mu, sigma, batch_labels)
            # Backward pass and optimization
            # Faster Way to zero gradients
            for param in model.parameters():
                param.grad = None
            
            loss.backward()
            optimizer.step()
            # Add Loss
            epoch_loss += loss.item()
        epoch_loss /= len(train_dataloader)
        print(f"Epoch: {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")
    #Eval
    print("Evaluating...")
    with torch.inference_mode():
        for batch_features, batch_labels in test_dataloader:
            # Forward pass
            mu, sigma = model(batch_features)
            # Compute the loss
            loss = crps(mu, sigma, batch_labels)
            # Add Loss
            test_loss += loss.item()
    test_loss /= len(test_dataloader)
    print(f"Test Loss: {test_loss:.4f}")
    torch.cuda.empty_cache()
    return test_loss

            

test_loss_list = []
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

for e in range(nreps):
    model = NNPP(INPUT_DIM=test_features.shape[1])
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
    loss = train(epochs=100,
                 train_features=train_features,
                 train_labels=train_targets,
                 test_features=test_features,
                 test_labels=test_targets,
                 model=model,
                 optimizer=optimizer,
                 crps=crps,
                 device=device)
    print(loss)
    test_loss_list.append(loss)

cuda:0
Training...
Epoch: 1/100, Loss: 1.9579
Epoch: 2/100, Loss: 1.3119
Epoch: 3/100, Loss: 1.2443
Epoch: 4/100, Loss: 1.2005
Epoch: 5/100, Loss: 1.1623
Epoch: 6/100, Loss: 1.1327
Epoch: 7/100, Loss: 1.1071
Epoch: 8/100, Loss: 1.0870
Epoch: 9/100, Loss: 1.0728
Epoch: 10/100, Loss: 1.0593
Epoch: 11/100, Loss: 1.0488
Epoch: 12/100, Loss: 1.0388
Epoch: 13/100, Loss: 1.0315
Epoch: 14/100, Loss: 1.0235
Epoch: 15/100, Loss: 1.0169
Epoch: 16/100, Loss: 1.0130
Epoch: 17/100, Loss: 1.0059
Epoch: 18/100, Loss: 1.0027
Epoch: 19/100, Loss: 0.9981
Epoch: 20/100, Loss: 0.9953
Epoch: 21/100, Loss: 0.9900
Epoch: 22/100, Loss: 0.9867
Epoch: 23/100, Loss: 0.9851
Epoch: 24/100, Loss: 0.9806
Epoch: 25/100, Loss: 0.9799
Epoch: 26/100, Loss: 0.9761
Epoch: 27/100, Loss: 0.9718
Epoch: 28/100, Loss: 0.9720
Epoch: 29/100, Loss: 0.9675
Epoch: 30/100, Loss: 0.9675
Epoch: 31/100, Loss: 0.9634
Epoch: 32/100, Loss: 0.9630
Epoch: 33/100, Loss: 0.9630
Epoch: 34/100, Loss: 0.9568
Epoch: 35/100, Loss: 0.9599
Epoch: 36/

KeyboardInterrupt: 